# Processing _Danish Energy Agency Technology Catalogue_ energy storage data for Mopo WP5

The aim of this workbook is to process the desired heat storage data from the
DEA technology catalogues into a format desired by WP5.

## Julia setup

This processing script is written in Julia, and some setup is required for it to run.

In [ ]:
## Activate (and set up) the required Julia environment

using Pkg # Julia package manager.
Pkg.activate(@__DIR__) # Activate the Julia environment in the folder this file is in (namely the `Project.toml`)
Pkg.instantiate() # Download and install the necessary dependencies.

# Load dependencies
using XLSX
using DataFrames
using Statistics
using CSV
using MopoHeatSectorDataProcessing

## Read DEA Data

In [ ]:
## Read the storage technology catalogue

raw_data = DataFrame(XLSX.readtable("input-data\\dea-technology-catalogues\\technology_datasheet_for_energy_storage.xlsx", "alldata_flat"))
first(raw_data, 3)

## Map DEA technologies to desired output technologies

In [ ]:
## Collect the set of all available technologies.

techs = Set(raw_data[:, :Technology])

# With only 15 technologies available, mapping is way easier than for heat generation.

### Large-scale hot water storage tanks

The `LT-hot-water-tank`, which I assume stands for "low temperature hot water tank"
or something? Regardless, its for district heating so large-scale in this catalogue.

In [ ]:
## Map district-heating-scale hot water tank.

DH_LT_hot_water_tank = filter(
    x -> contains(lowercase(x), "large-scale hot water tank"),
    techs
)

### Small-scale heat storage

Apparently, the `heat-storage` in the WP5 case refers to distributed
heat storage in buildings, essentially small-scale domestig hot water storage tanks.

In [ ]:
## Map distributed small-scale thermal storage.

distributed_hot_water_tanks = filter(
    x -> contains(lowercase(x), "small-scale hot water tank"),
    techs
)

## Map pit thermal storage

Just for good measure since we have it available.

In [ ]:
## Map pit thermal storage cause why not.

pit_thermal_storage = filter(
    x -> contains(lowercase(x), "thermal"),
    techs
)

## Calculate the desired parameters

Based on the _Conclusions_ in `examine_storage_data.ipynb`,
the rough plan for the parameters is as follows:

- `CAPEX Energy`: Likely some `Specific investment` or its `- energy component`.
- `CAPEX`: Likely `Specific investment` or its `- capacity component` similar to above.
- `FOM Energy`: Likely `Fixed O&M [EUR/MWhCapacity/year]` or similar.
- `FOM`: Likely `Fixed O&M [EUR/MW/year]` or similar.
- `VOM`: Easy `Variable O&M [EUR/MWh]`.
- `Charge Efficiency`: Charging/discharging efficiencies seem to be given as components of `Round trip efficiency`, e.g. `- Charge Efficiency [%]`. I can only hope they are available separately everywhere.
- `Discharge Efficiency`: Similar to above, but `- Discharge efficiency [%]`.
- `Energy-to-power ratio`: Ratio between energy content and discharging power. E.g. `Energy storage capacity for one unit [MWh]` divided by `Generating capacity for one unit [MW]`.
- `Currency`: EUR 2020, the `priceyear` column contains this information.
- `Metadata`: Add citation to the DEA catalogue in question.

**NOTE! In this data, `CAPEX Energy` and `CAPEX` are either or!**
Both contain all the investment costs for the entire plant
and are just alternative forms of the same costs!
As such, they are *NOT separate investments for the energy storage
and its charging infrastructure!
`FOM Energy` and `FOM` are similarly just two slightly different forms
of presenting the total fixed O&M costs.

In [ ]:
## Calculate the desired parameters

# Parameter name correction mapping table.
spelling_fixes = Dict(
    "[EUR/MWhCapacity/year)" => "[EUR/MWhCapacity/year]",
    "[MWh)" => "[MWh]",
    "[kWh)" => "[kWh]",
    "[MW)" => "[MW]",
    "[EUR/MW/year] - 1-2% of investment" => "[EUR/MW/year]",
    "[MW]*" => "[MW]"
)

# Preprocessing
cols = [:Technology, :par, :est, :year, :val] # Omit unnecessary cols
data = deepcopy(raw_data[!, cols])
data.est = lowercase.(data.est) # Avoid upper/lowercase issues
data.year = string.(data.year) # Avoid year stack/unstack issues
filter!(r -> r.est == "ctrl", data) # Filter our lower and upper estimates.
filter!(r -> typeof(r.val) != String, data) # Filter out string values.
map!(p -> replace(p, spelling_fixes...), data.par, data.par) # Fix bothersome misspellings.
unique!(data)
data = extrapolate_years( # Inter/extrapolate data to cover all years found within.
    data;
    extrapolation_years=[2015, 2018, 2019, 2020, 2030, 2040, 2050],
    combine=mean
)
data = unstack(data, :par, :val)

# Energy-to-power ratio (in hours), calculated first as this turns out to be useful for power-based CAPEX and FOM estimation.
data[!, :energy_to_power_ratio_h] = ( # Hours of energy output for large units.
    data[!, Symbol("Energy storage capacity for one unit [MWh]")]
    ./ data[!, Symbol("Output capacity for one unit [MW]")]
)
missing_inds = ismissing.(data.energy_to_power_ratio_h)
data[missing_inds, :energy_to_power_ratio_h] = ( # Hours of energy output for large units.
    data[missing_inds, Symbol("Energy storage capacity for one unit [kWh]")]
    ./ data[missing_inds, Symbol("Output capacity for one unit [kW]")]
)

# CAPEX_energy, Specific investment with the corresponding unit. Need to account for several different potential parameters
data[!, :CAPEX_energy_MEUR_GWh] = data[!, Symbol("Specific investment [MEUR/GWhCapacity]")]
missing_inds = ismissing.(data.CAPEX_energy_MEUR_GWh)
data[missing_inds, :CAPEX_energy_MEUR_GWh] = data[missing_inds, Symbol("Specific investment [EUR/kWh]")] # No scaling to MEUR/GWh required
missing_inds = ismissing.(data.CAPEX_energy_MEUR_GWh)
data[missing_inds, :CAPEX_energy_MEUR_GWh] = (
    data[missing_inds, Symbol("Specific investment [MEUR/MWh]")]
    .* 1000 # Requires scaling to reach MEUR/GWh
)

# CAPEX_power, `Specific investment` with the corresponding unit. Need to account for several params.
data[!, :CAPEX_power_MEUR_MW] = data[!, Symbol("Specific investment [MEUR/MW]")]
missing_inds = ismissing.(data.CAPEX_power_MEUR_MW)
data[missing_inds, :CAPEX_power_MEUR_MW] = (
    data[missing_inds, Symbol("Specific investment [EUR/kW]")]
    ./ 1000 # Requires scaling to reach MEUR/MW
)
missing_inds = ismissing.(data.CAPEX_power_MEUR_MW)
data[missing_inds, :CAPEX_power_MEUR_MW] = (
    data[missing_inds, :CAPEX_energy_MEUR_GWh] # Based on energy CAPEX
    ./ 1000 # Scaling to MEUR/MWh
    .* data[missing_inds, :energy_to_power_ratio_h] # Scaling based on energy-to-power ratio
)

# CAPEX_energy based on CAPEX power if it is still missing
missing_inds = ismissing.(data.CAPEX_energy_MEUR_GWh)
data[missing_inds, :CAPEX_energy_MEUR_GWh] = (
    data[missing_inds, :CAPEX_power_MEUR_MW] # Based on CAPEX power
    .* 1000 # Scaling to MEUR/GW
    ./ data[missing_inds, :energy_to_power_ratio_h] # Scaling based on energy-to-power ratio
)

# FOM_energy, `Fixed O&M`, although units vary. Small tanks also require specific processing.
data[!, :FOM_energy_EUR_GWh_y] = (
    data[!, Symbol("Fixed O&M [EUR/MWhCapacity/year]")]
    .* 1000 # Requires scaling to reach EUR/GWh/year
)
missing_inds = ismissing.(data.FOM_energy_EUR_GWh_y)
data[missing_inds, :FOM_energy_EUR_GWh_y] = (
    data[missing_inds, Symbol("Fixed O&M [EUR/tank/year]")] # Handle small-scale tanks
    ./ data[missing_inds, Symbol("Energy storage capacity for one unit [kWh]")]
    .* 1e6 # Scaling to EUR/GWh/y
)

# FOM_power, needs to handle multiple params, as well as estimate FOM costs for tanks.
data[!, :FOM_power_EUR_MW_y] = data[!, Symbol("Fixed O&M [EUR/MW/year]")]
missing_inds = ismissing.(data.FOM_power_EUR_MW_y)
data[missing_inds, :FOM_power_EUR_MW_y] = (
    data[missing_inds, Symbol("Fixed O&M [kEUR/MW/year]")]
    ./ 1000 # Scaling to EUR/MW/year
)
missing_inds = ismissing.(data.FOM_power_EUR_MW_y)
data[missing_inds, :FOM_power_EUR_MW_y] = ( # Estimate power fom costs for large tanks.
    data[missing_inds, :FOM_energy_EUR_GWh_y] # Based on energy FOM cost.
    ./ 1000 # Scaling to EUR/MWh/year
    .* data[missing_inds, :energy_to_power_ratio_h] # Scaling with "hours of energy", which is the ratio of energy to power.
)

# FOM_energy using FOM_power if not yet exists.
missing_inds = ismissing.(data.FOM_energy_EUR_GWh_y)
data[missing_inds, :FOM_energy_EUR_GWh_y] = (
    data[missing_inds, :FOM_power_EUR_MW_y] # Based on FOM_power
    .* 1000 # Scaling to EUR/GWh/y
    ./ data[missing_inds, :energy_to_power_ratio_h] # Scaling with energy-to-power ratio.
)

# VOM, handle small-scale tanks first to avoid electricity costs.
data[!, :VOM_EUR_MWh] = data[!, Symbol("- of which is other O&M costs [EUR/MWh]")]
missing_inds = ismissing.(data.VOM_EUR_MWh)
data[missing_inds, :VOM_EUR_MWh] = data[missing_inds, Symbol("Variable O&M [EUR/MWh]")]
missing_inds = ismissing.(data.VOM_EUR_MWh)
data[missing_inds, :VOM_EUR_MWh] = data[missing_inds, Symbol("Variable O&M [EUR/MWhoutput]")]

# Charge efficiency
data[!, :charge_efficiency_pu] = data[!, Symbol("- Charge efficiency [%]")] ./ 100 # Scaling to p.u.

# Discharging efficiency
data[!, :discharge_efficiency_pu] = data[!, Symbol("- Discharge efficiency [%]")] ./ 100 # Scaling to p.u.

# Storage losses
data[!, :storage_losses_pu_day] = (
    data[!, Symbol("Energy losses during storage [%/day]")]
    ./ 100 # Scaling to p.u.
)
missing_inds = ismissing.(data.storage_losses_pu_day)
data[missing_inds, :storage_losses_pu_day] = ( # Handle losses given hourly
    1 .- ( # Losses per day estimated from exponential hourly state change.
        1 .- data[missing_inds, Symbol("Energy losses during storage [%/hour]")]
        ./ 100 # Scaling to p.u.
    ) .^ 24 # Exponentiate hourly state change.
)
missing_inds = ismissing.(data.storage_losses_pu_day)
data[missing_inds, :storage_losses_pu_day] = ( # Estimate losses for pit thermal storage
    data[missing_inds, Symbol("Energy losses during storage [K/day]")]
    ./ (
        data[missing_inds, Symbol("Max. storage temperature, hot[⁰C]")]
        .- data[missing_inds, Symbol("Storage temperature, discharged [⁰C]")]
    )
)

# Lifetime
data[!, :lifetime_y] = data[!, Symbol("Technical lifetime [years]")]

# Currency is 2020 Euros based on `examine_storage_data.ipynb`
data[!, :currency] .= "2020 EUR"

# Metadata is the DEA technology catalogue
data[!, :metadata] .= "Technology data for energy storage - October 2018 - Updated April 2024, Danish Energy Agency, 2024"

# Drop unnecessary columns
param_cols = [:Technology, :est, :year]
params = [
    :CAPEX_energy_MEUR_GWh,
    :CAPEX_power_MEUR_MW,
    :FOM_energy_EUR_GWh_y,
    :FOM_power_EUR_MW_y,
    :VOM_EUR_MWh,
    :charge_efficiency_pu,
    :discharge_efficiency_pu,
    :storage_losses_pu_day,
    :energy_to_power_ratio_h,
    :lifetime_y,
    :currency,
    :metadata
]
data = data[!, vcat(param_cols, params)]
describe(data)

## Technology mapping

Map DEA technologies to desired Mopo WP5 technologies.
This is extremely straightforward for heat storaged.

In [ ]:
## Technology mapping

technology_mapping = Dict(
    "DH-LT-hot-water-tank" => DH_LT_hot_water_tank,
    "distributed-hot-water-tanks" => distributed_hot_water_tanks,
    "DH-pit-thermal-storage" => pit_thermal_storage
);

## Calculate aggregated parameters based on the above mappings

In [ ]:
## Form the desired technology parameter table

cols = [
    :technology,
    :year,
    :CAPEX_energy_MEUR_GWh,
    :CAPEX_power_MEUR_MW,
    :FOM_energy_EUR_GWh_y,
    :FOM_power_EUR_MW_y,
    :VOM_EUR_MWh,
    :charge_efficiency_pu,
    :discharge_efficiency_pu,
    :energy_to_power_ratio_h,
    :storage_losses_pu_day,
    :lifetime_y,
    :currency,
    :metadata
]
desired_data = DataFrame()
for (name, techs) in technology_mapping
    # Filter relevant technologies
    df = filter(
        r -> r.Technology in techs,
        data
    )
    isempty(df) && continue # Skip the rest of the loop if df is empty.
    # Calculate average properties
    df = unstack(
        combine(
            groupby(stack(df), [:year, :currency, :metadata, :variable]),
            :value => mean
        ),
        :variable,
        :value_mean
    )
    # Final formatting
    df.technology .= name
    append!(desired_data, df[!, cols])
end
describe(desired_data)

## Convert monetary units from 2020 EUR to 2025 EUR

For the purposes of Mopo, we need to convert the 2020 EUR into 2025 EUR.
This is done using the [harmonised index of consumer prices](https://ec.europa.eu/eurostat/cache/metadata/en/prc_hicp_esms.htm)
using the *annual averate rate of change `RCH_A_AVG`* for the full *Euro Area* for the years 2020-2024.

[Link to the data source](https://doi.org/10.2908/PRC_HICP_AIND).

In [ ]:
## Convert 2020 EUR to 2025 EUR using HICP

# Copy desired data to avoid messing with things on multiple executions.
final_data = deepcopy(desired_data)

# Annual rate of change in consumer prices for the Euro area, see data source above.
hicp_rch_a_avg_2020_2024 = [0.3, 2.6, 8.4, 5.4, 2.4] ./ 100 
# Calculate total conversion factor by multiplicatively applying the changes.
eur_2020_to_2025_conversion_factor = prod(1 .+ hicp_rch_a_avg_2020_2024)

# Apply monetary conversion to the desired fields
for field in [
    :CAPEX_energy_MEUR_GWh,
    :CAPEX_power_MEUR_MW,
    :FOM_energy_EUR_GWh_y,
    :FOM_power_EUR_MW_y,
    :VOM_EUR_MWh
]
    final_data[!, field] .*= eur_2020_to_2025_conversion_factor
end
# Set correct monetary unit and show
final_data.currency .= "2025 EUR"
describe(final_data)

## Inspect any potentially problematic missing data

No missing data it seems, nice!

## Format and export technology parameters

In [ ]:
## Format and export technology parameters

# Config
exportcols = [
    :technology,
    :year,
    :CAPEX_energy_MEUR_GWh,
    #:CAPEX_power_MEUR_MW,
    :FOM_energy_EUR_GWh_y,
    #:FOM_power_EUR_MW_y,
    :VOM_EUR_MWh,
    :charge_efficiency_pu,
    :discharge_efficiency_pu,
    :energy_to_power_ratio_h,
    :storage_losses_pu_day,
    :lifetime_y,
    :currency,
    :metadata
]
exportyears = string.([2030, 2040, 2050])
roundcols = exportcols[3:9]
dgts = 4 # Number of digits for rounding floats, storage losses need annoyingly many.

# Export
export_df = filter(r -> r.year in exportyears, final_data)
export_df[!, roundcols] = round.(export_df[!, roundcols]; digits=dgts)
CSV.write("output/heat_storage_technology_params.csv", export_df[!, exportcols])
